In [1]:
import sys

sys.path.append("..")

from itertools import product
import pandas as pd
import kornia.augmentation as K
import torch
import torch.nn as nn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

from src.models import get_model_by_name
from src.datasets.ucm import UCMMinimal
from src.utils import extract_features
from tqdm import tqdm

device = torch.device("cuda:0")

In [2]:
# Fit and save features
model_names = [
    "resnet50_pretrained_moco",
    "resnet18_pretrained_moco",
    "resnet50_pretrained_imagenet",
    "resnet50_randominit",
    "imagestats",
    "mosaiks_512_3",
]
rgbs = [True]
sizes = [64, 256]
normalization_methods = ["divide", "standardization"]
use_both_train_val = [True, False]

columns = {
    "model_name": [],
    "rgb": [],
    "size": [],
    "normalization": [],
    "use_train_val": [],
    "accuracy": [],
    "accuracy_scaler": [],
}

for model_name, rgb, size, normalization, use_both in tqdm(product(model_names, rgbs, sizes, normalization_methods, use_both_train_val), total=7*2*2*2):
    transforms = nn.Sequential(K.Resize(size)).to(device)
    dm = UCMMinimal(
        root="../data/ucm/",
        normalization_method=normalization,
        batch_size=64,
        num_workers=8,
        use_both_trainval=use_both,
    )
    dm.setup()


    model = get_model_by_name(model_name, rgb, device=device)

    x_train, y_train = extract_features(
        model, dm.train_dataloader(), device, transforms=transforms, verbose=False
    )
    x_test, y_test = extract_features(
        model, dm.test_dataloader(), device, transforms=transforms, verbose=False
    )

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(x_train, y_train)
    acc = knn_model.score(x_test, y_test)

    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(x_train, y_train)
    acc_scaler = knn_model.score(x_test, y_test)

    columns["model_name"].append(model_name)
    columns["rgb"].append(rgb)
    columns["size"].append(size)
    columns["normalization"].append(normalization)
    columns["use_train_val"].append(use_both)
    columns["accuracy"].append(acc)
    columns["accuracy_scaler"].append(acc_scaler)

 86%|████████▌ | 48/56 [03:35<00:35,  4.49s/it]


In [3]:
df = pd.DataFrame(columns)

In [4]:
df

,model_name,rgb,size,normalization,use_train_val,accuracy,accuracy_scaler
0,resnet50_pretrained_moco,True,64,divide,True,0.721429,0.600000
1,resnet50_pretrained_moco,True,64,divide,False,0.688095,0.550000
2,resnet50_pretrained_moco,True,64,standardization,True,0.333333,0.340476
3,resnet50_pretrained_moco,True,64,standardization,False,0.304762,0.309524
4,resnet50_pretrained_moco,True,256,divide,True,0.878571,0.888095
5,resnet50_pretrained_moco,True,256,divide,False,0.854762,0.854762
6,resnet50_pretrained_moco,True,256,standardization,True,0.535714,0.585714
7,resnet50_pretrained_moco,True,256,standardization,False,0.507143,0.557143
8,resnet18_pretrained_moco,True,64,divide,True,0.633333,0.602381
9,resnet18_pretrained_moco,True,64,divide,False,0.633333,0.576190


In [6]:
df.to_csv("../results/ucm_table_results.csv")